##**Timeline**
- Looked at previous dashboard code to learn what was used and what is needed (libraries and hardware)
- Made an official Formula SAE GitHub account to contain and log changes we made to any code that we used
- Made a mock-up tachometer to experiment on how to improve it and get more experience with working with Arduino
- Experimented with start-up animations for the tachometer
- Modified previous dashboard code to omit code that is no longer needed for the new dashboard (such as: 4 digit 14-segment LED displays, tachometer LED-strip clear functions)
- Experimented with a PWM driver for controlling brightness on the dashboard display and drive a 7-segment RGB 1" display for the gear position indicator
- Tested 2 warning LEDs for engine temperature and engine oil pressure with PWM driver
- Acquired 7-segment RGB 1" display from Adafruit. Soldered 7 wires connected to PWM driver to test functionality
- Acquired 7-segment RGB 1" display with NeoPixel support from RGBDigit. Simplifies amount of pins required and code (only 3 pins and uses NeoPixel library). No need for PWM driver to control 7-segment display
- Made a mock-up gear position shifter with color matching to tachometer
- Updated dashboard code to support PWM driver and 7-segment display. Rough draft ready for testing
- Acquired main Arduino board, CAN BUS shield, and custom power PCB shield for testing new dashboard code. Made prototype of all wiring to test with dynamometer. See **Dyno To Do Procedure**
- Wired up Arduino to final dashboard. Tested functionality and adjusted values
- Added an input to the neutral engage line.
- Experimented with Timer1 and interrupts to have warning LEDs blink. Found out that NeoPixels disable and re-enable interrupts with each function call. Would be hard to implement with NeoPixels so removed timer and interrupt
- Reprogrammed ECU to only send neccessary CAN data for dashboard and AiM
- LEDs can blink just fine with each update call
- Added day and evening brightness control that corresponds to calibration switch
- Added an idle or "sleep" mode when engine is not running

##**Hardware**
- Arduino Uno
- Custom PCB power shield
- 2 x 10k ohm resistor
- 1 x 330 ohm resistor

- [CAN-BUS Shield v1.2](https://www.seeedstudio.com/CAN-BUS-Shield-V1.2-p-2256.html)
  - Libraries: mcp_can.h, SPI.h
  - Requires digital pins 9-13

- 2 x [Adafruit NeoPixel Stick 8 x 5050 RGB LED](https://www.adafruit.com/product/1426)
  - Libraries: Adafruit_NeoPixel.h library
  - Operational Frequency: 800 kHz datastream
  - Can use any digital output pin
  - 300-500 ohm resistor on data line
  
- [Adafruit 12-chanel 16-bit PWM LED driver SPI](https://www.adafruit.com/product/1455)
  - Libraries: Adafruit_TLC59711.h, SPI.h
  - 9600 baud rate
  - Any digital pin can be used for the SPI data output and SPI clock
  - 12V on V+
  
- [RGBDigit 1" 7-segment RGB Display](https://www.rgbdigit.com/shop/index.php?id_product=8&controller=product&id_lang=1)
  - Favored using this version for its simplicity
  - Built in 330 ohm resistor on data pin

##**Code Documentation**

##Libraries

In [ ]:
// Libraries:
#include <mcp_can.h>
#include <SPI.h>
#include <Adafruit_NeoPixel.h>
#include <Adafruit_TLC59711.h>

- mcp_can: used for allowing the CAN-BUS Shield to communicate between the CAN line and the arduino board. Publicly available from Seeed Studio.
- SPI: library for communication between several hardware. Included with Arduino.
- Adafruit_Neopixel: all NeoPixels rely on this library to function (LED strip and 7-segment display). Publicly available from Adafruit.
- Adafruit_TLC59711: handles communication with the TLC59711 PWM driver. Publicly available from Adafruit.

##Defines and Initializations
Non-variable and final values should be defined here as #defines so it can be changed easily if need be and can't be changed while running the code.

In [ ]:
// LED NeoPixel strip parameters
#define SPI_CS_PIN 9                                  // Declares D9 as CS for Seeed's CAN-BUS Shield.
#define stripLength 16                                // NeoPixel LED strip length of 16
#define stripDataPin 5                                // Declares D5 for strip data.
#define blinkInterval 50                              // flashing blink interval

// Segment display paramaters
#define segmentLength 7                               // Segment display length of 7
#define segmentDataPin 3                              // D3 for segment display data

// Neutral digital read pin
#define neutralPin 6

// Absolute brightness control for LEDs
#define stripBrightness 200                                // 0 = off, 255 = fullbright
#define segBrightness 255                                  // brightness settings here set absolute values

// TLC59711 parameters
#define NUM_TLC59711 1 // amount of chips connected
#define data 8         // PWM data pin (any digital pin)
#define clock 7        // PWM clock pin (any digital pin)
#define ENG_LED 0      // engine coolant temp LED R0
#define OIL_LED 1      // EOP LED G0
int DUTY = 5;          // values from 0 to 100% representing duty cycle

// Engine parameters                                         CHANGE VALUES HERE
#define low_rpm 2200
#define high_rpm 10000
#define HIGH_ENG_TEMP 100
#define UNSAFE_ENG_TEMP 105
#define HIGH_EOP 55

// Library Initializations:
MCP_CAN CAN(SPI_CS_PIN);      // Sets CS pin.
Adafruit_NeoPixel strip = Adafruit_NeoPixel(stripLength, stripDataPin, NEO_GRB + NEO_KHZ800);      // Configures the NeoPixel Strip for 16 LEDs.
Adafruit_NeoPixel seg = Adafruit_NeoPixel(segmentLength, segmentDataPin, NEO_GRB + NEO_KHZ800);     // Configures 7-segment display for 8(actually 7) segments

uint16_t PWM_LEVEL = map(DUTY, 0, 100, 0, 65535); // (DUTY/100) * 65535; PWM value ranges from 0 to 65535
Adafruit_TLC59711 LED = Adafruit_TLC59711(NUM_TLC59711, clock, data); // initialization with parameters (chip amount, clock pin, data pin)

**PIN 9** is the chip select pin needed for SPI to communicate with the CAN-BUS Shield. **PIN 10** can be optionally configured to be the chip select pin physically on the CAN-Bus Shield (see shield documentation). **PINs 11-13** are also needed for SPI communication. 16 NeoPixel LEDs are used so it must be defined for the LED strip initialization. **PIN 5** was chosen as the data pin for the LED strip but can be any other free digital pin. blinkInterval is the shift warning flash interval counted in milliseconds with an internal arduino function used as a timer.

The 7-segment display has seven LEDs since the dot has none. **PIN 3** was chosen as the data pin for the segment display but can be any other free digital pin.

A digitial pin is used as input to read the neutral state. **PIN 6** was chosen and has a pull-up resistor to default to HIGH when a neutral signal is not active LOW.

Brightness is the initial brightness setting for both NeoPixel strip and segment display but colors can be set to whatever brightness later on. Brightness is the absolute brightness setting for NeoPixel LEDs. Relative brightness can be set in the RGB colors.

Number of TLC59711 chips chained together must be defined for initialization. Data and clock pins can be any free digital pins. The PWM driver chip has 12 channels so 12 different chains of LEDs can be controlled. On the chip, pins are labeled RGB with numbers 0-3. Channels are counted up starting from 0. For example, **R0** is channel 0, **G0** is channel 1, **R1** is channel 3, etc. In this case, the engine warning LED is wired to R0 and the EOP LED is wired to G0. A duty percentage is defined for ease of brightness/PWM control.

Engine parameters are defined with the minimum RPM level and maximum RPM shift point for the tachometer. Engine temperature and EOP warnings are set to turn on the PWM LEDs when values exceed the defined values.

mcp_can library is initialized with whatever the chip select pin number is. Typically, and in this case, it is **PIN 9**.
The NeoPixel library is initialized with (LED amount, data pin, LED type and timing). Both NeoPixel strip and segment are RGB and have 800kHz data rates. **NEO_GRB + NEO_KHZ800** and others are defined in the .h file of the library.

The PWM driver is 16 bits so it can control PWM signals up to 2^16-1. For ease, the numbers are mapped to 0-100 as the duty cycle. The TLC59711 chip is then initialized with (number of chips chained, clock pin, data pin).

##Variable Definitions

In [ ]:
// Define LED colors, these are relative to the absolute brightness set above; MAY CHANGE BRIGHTNESS
uint32_t greenStripDay = strip.Color(0, 50, 0),       // daytime brightness
         yellowStripDay = strip.Color(50, 50, 0),
         redStripDay = strip.Color(50, 0, 0),
         redMaxStripDay = strip.Color(150, 0, 0),
         
         greenStripNight = strip.Color(0, 10, 0),     // lower brightness for evening
         yellowStripNight = strip.Color(10, 10, 0),   
         redStripNight = strip.Color(10, 0, 0),
         redMaxStripNight = strip.Color(30, 0, 0),
         color[4] = {greenStripDay, yellowStripDay, redStripDay, redMaxStripDay};


// 7 segment display colors
uint32_t redSegDay = seg.Color(255, 0, 0),               // daytime brightness
         yellowSegDay = seg.Color(255, 255, 0),
         greenSegDay = seg.Color(0, 255, 0),

         redSegNight = seg.Color(30, 0, 0),              // evening brightness
         yellowSegNight = seg.Color(30, 30, 0),
         greenSegNight = seg.Color(0, 30, 0),

         redSeg = redSegDay,                             // initial display brightness
         yellowSeg = yellowSegDay,
         greenSeg = greenSegDay,
         segColor = greenSeg,
         prev_segColor = greenSeg;

// 7 segment display gear
int digitArray[7][7] = {                    //                            ___________      
                    {0, 0, 1, 0, 1, 0, 1},  // n                         |     0     |
                    {0, 1, 1, 0, 0, 0, 0},  // 1                         |  5     1  |
                    {1, 1, 0, 1, 1, 0, 1},  // 2                         |     6     |
                    {1, 1, 1, 1, 0, 0, 1},  // 3                         |  4     2  |
                    {0, 1, 1, 0, 0, 1, 1},  // 4                         |     3     |
                    {1, 0, 1, 1, 0, 1, 1},  // 5                         |___________|
                    {1, 0, 1, 1, 1, 1, 1}   // 6                         
                    };

int digitCircle[6][7] = {
                    {1, 0, 0, 0, 0, 0, 0},
                    {0, 1, 0, 0, 0, 0, 0},
                    {0, 0, 1, 0, 0, 0, 0},
                    {0, 0, 0, 1, 0, 0, 0},
                    {0, 0, 0, 0, 1, 0, 0},
                    {0, 0, 0, 0, 0, 1, 0}
                    };

int shiftPT[2] = {low_rpm, high_rpm};   // sets lower and upper limits of RPMs to display

int prev_range = 10,        // needed for tachometer updating, arbitrary number
    prev_gear = 10,         // needed for gear position updating, arbitrary number
    ledStages[2] = {5, 11}, // this is where each stage of the led strip is set. i.e. from ledStages[0] and ledStages[1] is stage one and so on
    led_pos = 0;            // position of LEDs in sleep mode
long prevBlinkTime = 0;     // timer for warning flash interval

bool warningState = false,  // blink state for shift point flashing
     eng_state = false,     // blink state for engine warning LED
     eop_state = false,     // blink state for oil pressure warning LED
     pos_state = false,     // move LEDs left or right in sleep mode
     enable_state = false;  // flag for sleep mode

LED colors are predefined for use on the NeoPixel strip and segment display. Colors are defined in RGB format and require an **unsigned 32 bit** variable. Relative brightness of colors can be controlled with the RGB value. RGB values range from 0-255 (R,G,B).

A 2D array is used to hold the predefined gear positions to display on the segment display. Number of rows is for the amount of gears defined, and number of columns is for the seven LEDs on the segment display. On the right in the comments is the segment display pictured with segments associated with the column number. Other 2D arrays can be defined for custom usage such as startup animations.

Shift point parameters for the tachometer, setting lowest RPM and upper RPM shift warning.

Previous tachometer and gear variables are for updating both only when there's a change to reduce unnecessary flickering updates. These initial values are arbitrary.

ledstages are to separate the tachometer colors from green to yellow to red depending on the RPM.

led_pos is to keep track of the "moving" LEDs when running sleep mode.

These set of boolean values keep track of what state the LEDs are in, flagging the program whether they should be on or off when blinking or direction of movement. enable_state being the exception where it prevents a condition from running in sleep mode.

##Setup

In [ ]:
void setup()
{
  //Serial.begin(115200);     // comment out all Serials and Strings when not testing
  do
  {
    if (CAN_OK == CAN.begin(CAN_1000KBPS))  // initializes CAN-BUS Shield at specified baud rate.
    {
      //Serial.println("CAN-BUS Shield Initialized!");
      //is_CBS_init = true;
      break;
    }
    else
    {
      //Serial.println("CAN-BUS Shield FAILED!");
      //Serial.println("Retry initializing CAN-BUS Shield.");
      delay(1000);
    }
  }
  while(true);

  pinMode(neutralPin, INPUT); // sets neutralPin (pin 6) as input
  
  // begin NeoPixel strip
  strip.begin();
  strip.setBrightness(stripBrightness);
  strip.clear();
  strip.show();

  // begin segment display
  seg.begin();
  seg.setBrightness(segBrightness);
  seg.clear();
  seg.show();

  LED.begin();
  
  startupAnimation();
}

setup() is the initial code to run when the arduino turns on. It first attempts to initialize the CAN-BUS Shield at a specified baud rate defined in mcp_can.h. If successful, it breaks out of the do while loop. If not, it keeps looping forever until it's successful.

pinMode sets whatever the neutralPin to read as input, since Arduino defaults pins to output.

Initialize NeoPixel strip by calling .begin(), setting the brightness .setBrightness(), clearing it .clear(), and displaying .show(). The setBrightness command should only be called once for initialization. It should never be used to update the brightness anywhere else since it will mess up the timing on NeoPixel LEDs. The segment display is initalized the same way as the NeoPixel strip.

The PWM driver only needs the .begin() command to initialize.

startupAnimation() calls the function to display a startup animation on the dashboard everytime the ECU initially receives power (kill switch is not engaged).

##Loop

In [ ]:
void loop()
{
  unsigned char len = 0;
  unsigned char buf[4];
  if(CAN_MSGAVAIL == CAN.checkReceive()) //Checks for incoming data.
  {
    CAN.readMsgBuf(&len, buf);           //Reads incoming data. len: data length, buf[location]: actual data.
    unsigned int canId = CAN.getCanId();
    //String id = String(canId);
    //Serial.println(id);
    if (canId == 1536)
    {
      int rpmA = buf[0];
      int rpmB = buf[1];
      int ENGINE_RPM = ((rpmA * 256) + rpmB);
      ledStrip_update(ENGINE_RPM);
      //gearOFF(ENGINE_RPM);
      //String ALPHA_RPM = String(ENGINE_RPM);
      //Serial.println("RPM: " + ALPHA_RPM + "\n");
    }

    if (canId == 1550)
    {
      int gearA = buf[2];
      int gearB = buf[3];
      int GEAR = ((gearA * 256) + gearB - 2);
      gearShift_update(GEAR, segColor);
      //String ALPHA_GEAR = String(GEAR);
      //Serial.println("GEAR: " + ALPHA_GEAR + "\n");
    }
  
    if (canId == 1541)
    {
      int tempA = buf[2];
      int tempB = buf[3];
      int ENGINE_TEMP = ((tempA * 256) + tempB) / 10;
      TEMP_warning(ENGINE_TEMP);
      //String ALPHA_TEMP = String(ENGINE_TEMP);
      //Serial.println("ENG TEMP: " + ALPHA_TEMP + "\n");
    }

    if (canId == 1544)
    {
      int eopA = buf[0];
      int eopB = buf[1];
      int INT_EOP = (((eopA * 256) + eopB) * 0.0145037738); // millibars to PSI conversion
      EOP_warning(INT_EOP);
      //String ALPHA_EOP = String(INT_EOP);
      //Serial.println("EOP: " + ALPHA_EOP + "\n");
    }

    if (canId == 1546)
    {
      int enableA = buf[0];
      int enableB = buf[1];
      int calA = buf[2];
      int calB = buf[3];
      int ENABLE = ((enableA * 256) + enableB);
      int CAL = ((calA * 256) + calB);
      sleep(ENABLE);
      brightness_ctrl(CAL);
      //String ALPHA_EN = String(ENABLE);
      //Serial.println("ENABLE: " + ALPHA_EN + "\n");
      //String ALPHA_CAL = String(CAL);
      //Serial.println("CAL: " + ALPHA_CAL + "\n");
    }
  }
}

loop() loops whatever code inside it forever while the Arduino is on and running.

The len variable is initialized as an unsigned char (uint8_t) so it can hold the data length.
The buf array holds four unsigned chars for the CAN message frame. The ECU CAN sends 8 data bytes per frame, but we happen to only need the first four. Four slots of data are sent in a single frame, with each slot consisting of 16 bits of data, with the high byte sent first. Conversions will need to be done to get the proper value, hence the first byte of data being multiplied by 256.

If there's a message available from the CAN, then run the code, otherwise, skip and loop around. CAN_MSGAVAIL is a defined value from the mcp_can library and if it matches .checkReceive() value, then continue.

.readMsgBuf() reads the incoming data. The first parameter requires the memory address of the len variable since the function dereferences it. Second parameter is the variable to hold the incoming data.

The CAN message frame ID is then requested and placed into a variable so we know which piece of data we're receiving. See ECU CAN layout Excel file in Electrical/Documentation/Dashboard Google Drive folder.

Based on the CAN ID number, if it matches the number (in decimal), then run the specified lines of code that corresponds to the data.

canID == 1536 is the Engine RPM and the data is held in the first two array positions. It is then converted to a decimal value in RPM. A function is then called to update the tachometer (NeoPixel strip).

canID == 1550 is the gear position data, with data being held in the last two positions of the buffer array. The upper and lower bytes are converted and added to get the gear number with a -2 at the end. This is because in the ECU, 0 is undefined, 1 is reverse, 2 is neutral, 3 is 1st gear, and so on. In the arduino, we defined 0 as neutral and 1 and 1st gear, and etc. A gear update function is called to update the segment display with the gear position.

canID == 1541 is the engine coolant temperature data, with data being located in the last two positions of the array. Number is divided by 10 to get correct value in Celsius. A temperature warning function is called to see if the temperature warning LED needs to be toggled or flashed.

canID == 1544 is the engine oil pressure data, with data inside the first two bytes of the array. The value is converted to a decimal number which is in millibars. It is then converted to PSI. It then sends the value to an oil pressure warning function to see if the LED should be toggled or flashed.

canID == 1546 is the engine enable status and the calibration switch data. Engine enable indicates when the engine is on or off. Calibration value is the position the calibration switch is in minus one. Both values are converted normally. The cal_switch function is called to adjust day or evening brightness of the LEDs. The sleep function is called with the engine status value to determine whether to run the animation or not.

##LED Strip Update (Tachometer)

In [ ]:
void ledStrip_update(int rpm)   // tachometer function
{
  int LED_RPM = rpm;
  unsigned long currentMillis = millis();   // get current timer for flashing condition
  if (LED_RPM >= shiftPT[0] && LED_RPM < shiftPT[1]) // if the RPM is between the lowest RPM and the shift point
  {
    // map the RPM values from 0 to 16 (really 15 since the shift point and beyond is handled below) and constrain the range
    int rpmMapped = map(LED_RPM, shiftPT[0], shiftPT[1], 0, 16);
    int rpmConstrained = constrain(rpmMapped, 0, 16);

    if (prev_range != rpmConstrained)                // this makes it so we only update the LED when the range changes so we don't readdress the strip every reading
    {
      prev_range = rpmConstrained;
      strip.clear();
      for (int ledNum = 0; ledNum <= rpmConstrained; ledNum++) // rpmConstrained makes ledNum refer to one LED too much on the light strip since the light strip is indexed from 0,
                                                               // causes brief moment of max rpm before flashing to change gear
      {
        if (ledNum <= ledStages[0])                            // green
        {
          strip.setPixelColor(ledNum, color[0]);
          segColor = greenSeg;
        }
        else if (ledNum > ledStages[0] && ledNum <= ledStages[1])   // yellow
        {
          strip.setPixelColor(ledNum, color[1]);
          segColor = yellowSeg;
        }
        else if (ledNum > ledStages[1] && ledNum < stripLength)     // red
        {
          strip.setPixelColor(ledNum, color[2]);
          segColor = redSeg;
        }
      }
      strip.show();
    }
  }
  else if (LED_RPM >= shiftPT[1]) // RPM at or above shift point, blinks the LEDS on and off with no blocking delay
  {
    prev_range = 16;
    if (currentMillis - prevBlinkTime > blinkInterval)    // millis eventually overflows but only after 50 days powered on so abs value is not needed here
    {
      prevBlinkTime = currentMillis;

      if (warningState == false)
      {
        warningState = true;
        for (int i = 0; i < stripLength; i++)
        {
          strip.setPixelColor(i, color[3]);
        }
        strip.show();
      } 
      else
      {
        strip.clear();
        strip.show();
        warningState = false;
      }
    }
  }
  else
  {
    if (prev_range != 10)
    {
      prev_range = 10;
      strip.clear();
      strip.show();
    }
  }
}

This function updates the LED strip according to the RPM value sent from the ECU. First it gets the current time since the board was running in milliseconds for the blinking function. It then checks if the RPM is between the lowest and highest RPM values for normal LED updating.

In this conditional, it first maps(interpolates) the RPM value to a value between 0 to 16. 16 because we want the last LED to be on before the blinking shift point. It is then constrained so the value will not exceed below 0 or above 16 for extra precaution. The LED strip is only updated if it's not the same value as before to avoid flickering the LEDs that are currently on and save time. A for loop is then used to display the number of LEDs corresponding based on the mapped and constrained value and depending on the value, changes the color. The color of the gear display is changed here to match with the tachometer.

If the RPM value is above the shift point RPM, then it enters a special condition to where all the strip LEDs blink. It then checks if the current time subtracted by the previous time is greater than the blink interval (in this case 50 ms), then it should either turn the LEDs on or off. A conditional flag is set to remember the previous state the LEDs were in and act accordingly. All the LEDs are either on and a brighter red, or off.

An else conditional is set as a catchall to reset the previous number of LED value and clear the LED strip.

##7-segment Display Update (Gear Position)

In [ ]:
void gearShift_update(int gear, uint32_t segColor) // update and display gear number on segment display
{
  //Serial.println("Neutral:" + digitalRead(neutralPin));
  if (digitalRead(neutralPin) == LOW)
  {
    gear = 0;
    segColor = greenSeg;
    seg.clear();
    seg.show();
    for(int i = 0; i <= 6; i++)
    {
      if(digitArray[gear][i])
        seg.setPixelColor(i, segColor);
    }
    seg.show();
  }

  else if (prev_segColor != segColor || gear != prev_gear) { // short circuit evaluation
    prev_segColor = segColor;
    prev_gear = gear;
    seg.clear();
    seg.show();
    for(int i = 0; i <= 6; i++)
    {
      if(digitArray[gear][i])
        seg.setPixelColor(i, segColor);
    }
    seg.show();
  }
}

In this function, the 7-segment display or gear indicator is updated. First it checks if the signal from the neutral engage is an active low. If it is, then make the display show neutral.

It then checks if the color of the display or gear position has changed from the previous value. This is to prevent flickering and saves time evaluating. More often, the color of the segment changes than the gear position so that gets evaluated first. The code then updates the segment display with the appropriate color and gear number.

The for loop runs through the column length of a predefined 7-segment 2D array. It uses the gear number as the row number and the loop counter as the column number. If there's a "1" in that array element, then turn that particular segment on with the corresponding color value. Otherwise, do nothing and continue the loop.

##Engine Coolant Temperature Warning LED

In [ ]:
void TEMP_warning(int engine_temp) // turn LED on when engine temp is outside safe parameters
{
  if (engine_temp >= HIGH_ENG_TEMP && engine_temp < UNSAFE_ENG_TEMP)
  {
    LED.setPWM(ENG_LED, PWM_LEVEL);
  }
  else if (engine_temp >= UNSAFE_ENG_TEMP)
  {
    if (eng_state == false)
    {
      LED.setPWM(ENG_LED, PWM_LEVEL);
      eng_state = true;
    }
    else
    {
      LED.setPWM(ENG_LED, 0);
      eng_state = false;
    }
  }
  else
  {
    LED.setPWM(ENG_LED, 0);
  }
  LED.write();
}

This function toggles or blinks the temperature warning LED. If the temperature is between the two set parameters, then have a steady LED come on.

If the temperature is above the upper parameter, have the LED blink. A conditional flag (eng_state) is used to allow the LED to blink.

If none of these conditions are met, then have the LED be off.

##Engine Oil Pressure Warning LED

In [ ]:
void EOP_warning(int EOP) // turn LED on when oil pressure is outside safe parameters
{
  if (EOP >= HIGH_EOP)
  {
    LED.setPWM(OIL_LED, PWM_LEVEL);
  }
  else if (EOP <= 8)
  {
    if (eop_state == false)
    {
      LED.setPWM(OIL_LED, PWM_LEVEL);
      eop_state = true;
    }
    else
    {
      LED.setPWM(OIL_LED, 0);
      eop_state = false;
    }
  }
  else
  {
    LED.setPWM(OIL_LED, 0);
  }
  LED.write();
}

Similar to the temperature warning function, this turns on or off the oil pressure warning LED under certain conditions.

If the oil pressure is too high, have the LED be steady on.

If it's too low, the LED should blink. A conditional flag (eop_state) is used to let the LED blink.

Otherwise, turn off the LED.

##Brightness Control

In [ ]:
void brightness_ctrl(int CAL)
{
  if (CAL >= 0 && CAL <= 3) // day time brightness
  {
    color[0] = greenStripDay;
    color[1] = yellowStripDay;
    color[2] = redStripDay;
    color[3] = redMaxStripDay;

    redSeg = redSegDay;
    yellowSeg = yellowSegDay;
    greenSeg = greenSegDay;
    
    DUTY = 5;
    PWM_LEVEL = map(DUTY, 0, 100, 0, 65535);
  }

  else  // night time brightness
  {
    color[0] = greenStripNight;
    color[1] = yellowStripNight;
    color[2] = redStripNight;
    color[3] = redMaxStripNight;

    redSeg = redSegNight;
    yellowSeg = yellowSegNight;
    greenSeg = greenSegNight;
    
    DUTY = 1;
    PWM_LEVEL = map(DUTY, 0, 100, 0, 65535);
  }
}

By using the calibration switch positions, the drivers can control the brightness for either daytime or evening driving.

From calibration values 0 to 3, have the displays and LEDs be very bright. The strip LEDs, segment display, and warning LEDs are set to bright levels.

For the rest of the values, dim the brightness significantly.

##Idle/Sleep Animation

In [ ]:
void sleep(int ENABLE)  // idle animation when engine is not running
{
  if (ENABLE == 0)
  {
    strip.clear();
    
    if (led_pos <= ledStages[0])
      strip.setPixelColor(led_pos, color[0]);
      
    else if (led_pos > ledStages[0] && led_pos <= ledStages[1])
      strip.setPixelColor(led_pos, color[1]);
      
    else if (led_pos > ledStages[0] && led_pos < stripLength)
      strip.setPixelColor(led_pos, color[2]);
    
    strip.show();

    if (pos_state == false)
    {
      led_pos++;
      if (led_pos == 15)
        pos_state = true;
    }
    else if (pos_state == true)
    {
      led_pos--;
      if (led_pos == 0)
        pos_state = false;
    }

    enable_state = false;
  }

  else if (ENABLE == 1 && enable_state == false)
  {
    strip.clear();
    strip.show();
    enable_state = true;
  }
}

This function is entirely optional for the dashboard code but it provides an aesthetic while the engine is off and to show that the LED strip still functions.

If the engine is detected to be off, move an LED on the NeoPixel strip left to right and right to left repeatedly.

If the engine is reported on, clear the LED strip.

The enable_state flag is so that the engine on conditional in this function only runs once. If such flag isn't implemented, it would run the else if conditional while the engine is on repeatedly, constantly and unintentionally clearing the LED strip.

##Startup Animation

In [ ]:
void startupAnimation()
{
  LED.setPWM(ENG_LED, PWM_LEVEL);
  LED.setPWM(OIL_LED, PWM_LEVEL);
  LED.write();
  segColor = greenSeg;
  for(int h = 0; h <= 15; h++)
  {
    for(int i = 0; i <= 5; i++)
    {
      for(int j = 0; j <= 6; j++)
      {
        if(digitCircle[i][j])
          seg.setPixelColor(j, segColor);
      }
      seg.show();
      delay(30);
      seg.clear();
    }

    if (h <= ledStages[0])
    {
      strip.setPixelColor(h, color[0]);
      segColor = greenSeg;
    }
    else if (h > ledStages[0] && h <= ledStages[1])
    {
      strip.setPixelColor(h, color[1]);
      segColor = yellowSeg;
    }
    else if (h > ledStages[1] && h < stripLength)
    {
      strip.setPixelColor(h, color[2]);
      segColor = redSeg;
    }
    strip.show();
  }
  seg.clear();
  seg.show();
  LED.setPWM(ENG_LED, 0);
  LED.setPWM(OIL_LED, 0);
  LED.write();
  delay(500);
  strip.clear();
  strip.show();
  delay(1000);
}

Here, the startup animation is defined. It runs every time the ECU is powered on (kill switch is disengaged). It simply turns on both warning LEDs, "spins" the segment display, and make the LED strip look like a loading bar. The segment display spins with a 30ms delay and after every cycle, it adds an LED to the strip until it's full. Then after the loop is done, it clears the whole dashboard.

###**Entirety of Dashboard Code**

In [ ]:
/*
   Version Notes:
    Beta: Only reads engine RPM from ECU and prints to serial monitor.
    v1.0: Added support for Adafruit NeoPixel as tachometer/shift LEDs.
    v2.0: Added support for Adafruit Alphanumeric Backpacks.
    v3.0: Added support for Gear Position from ECU and prints to Alphanumeric Backpack.  Fixed brackets and comments for typos and clarity.
    v4.0: Added support for Wheel Speed and Coolant Temperature from ECU.  Can print to Alphanumeric Backpack.  Build for unveiling.
    v5.0: Added support for Oil Pressure from ECU.  Can print to Alphanumeric Backpack.
    v5.1: Bug fixes.
    v6.0: Removed various functions such as 14-segment displays. Added support for a 7-segment display and PWM driver.
*/

// Libraries:
#include <mcp_can.h>
#include <SPI.h>
#include <Adafruit_NeoPixel.h>
#include <Adafruit_TLC59711.h>

// LED NeoPixel strip parameters
#define SPI_CS_PIN 9                                  // Declares D9 as CS for Seeed's CAN-BUS Shield.
#define stripLength 16                                // NeoPixel LED strip length of 16
#define stripDataPin 5                                // Declares D5 for strip data.
#define blinkInterval 50                              // flashing blink interval

// Segment display paramaters
#define segmentLength 7                               // Segment display length of 7
#define segmentDataPin 3                              // D3 for segment display data

// Neutral digital read pin
#define neutralPin 6

// Absolute brightness control for LEDs
#define stripBrightness 200                                // 0 = off, 255 = fullbright
#define segBrightness 255                                  // brightness settings here set absolute values

// TLC59711 parameters
#define NUM_TLC59711 1 // amount of chips connected
#define data 8         // PWM data pin (any digital pin)
#define clock 7        // PWM clock pin (any digital pin)
#define ENG_LED 0      // engine LED R0
#define OIL_LED 1      // EOP LED G0
int DUTY = 5;          // values from 0 to 100% representing duty cycle

// Engine parameters                                         CHANGE VALUES HERE
#define low_rpm 2000
#define high_rpm 12000
#define HIGH_ENG_TEMP 100
#define UNSAFE_ENG_TEMP 105
#define HIGH_EOP 55

// Library Initializations:
MCP_CAN CAN(SPI_CS_PIN);      // Sets CS pin.
Adafruit_NeoPixel strip = Adafruit_NeoPixel(stripLength, stripDataPin, NEO_GRB + NEO_KHZ800);      // Configures the NeoPixel Strip for 16 LEDs.
Adafruit_NeoPixel seg = Adafruit_NeoPixel(segmentLength, segmentDataPin, NEO_GRB + NEO_KHZ800);     // Configures 7-segment display for 8(actually 7) segments

uint16_t PWM_LEVEL = map(DUTY, 0, 100, 0, 65535); // (DUTY/100) * 65535; PWM value ranges from 0 to 65535
Adafruit_TLC59711 LED = Adafruit_TLC59711(NUM_TLC59711, clock, data); // initialization with parameters (chip amount, clock pin, data pin)

// Define LED colors, these are relative to the absolute brightness set above; MAY CHANGE BRIGHTNESS
uint32_t greenStripDay = strip.Color(0, 50, 0),       // daytime brightness
         yellowStripDay = strip.Color(50, 50, 0),
         redStripDay = strip.Color(50, 0, 0),
         redMaxStripDay = strip.Color(150, 0, 0),
         
         greenStripNight = strip.Color(0, 10, 0),     // lower brightness for evening
         yellowStripNight = strip.Color(10, 10, 0),   
         redStripNight = strip.Color(10, 0, 0),
         redMaxStripNight = strip.Color(30, 0, 0),
         color[4] = {greenStripDay, yellowStripDay, redStripDay, redMaxStripDay};


// 7 segment display colors
uint32_t redSegDay = seg.Color(255, 0, 0),               // daytime brightness
         yellowSegDay = seg.Color(255, 255, 0),
         greenSegDay = seg.Color(0, 255, 0),

         redSegNight = seg.Color(30, 0, 0),              // evening brightness
         yellowSegNight = seg.Color(30, 30, 0),
         greenSegNight = seg.Color(0, 30, 0),

         redSeg = redSegDay,                             // initial display brightness
         yellowSeg = yellowSegDay,
         greenSeg = greenSegDay,
         segColor = greenSeg,
         prev_segColor = greenSeg;

// 7 segment display gear
int digitArray[7][7] = {                    //                            ___________      
                    {0, 0, 1, 0, 1, 0, 1},  // n                         |     0     |
                    {0, 1, 1, 0, 0, 0, 0},  // 1                         |  5     1  |
                    {1, 1, 0, 1, 1, 0, 1},  // 2                         |     6     |
                    {1, 1, 1, 1, 0, 0, 1},  // 3                         |  4     2  |
                    {0, 1, 1, 0, 0, 1, 1},  // 4                         |     3     |
                    {1, 0, 1, 1, 0, 1, 1},  // 5                         |___________|
                    {1, 0, 1, 1, 1, 1, 1}   // 6                         
                    };

int digitCircle[6][7] = {
                    {1, 0, 0, 0, 0, 0, 0},
                    {0, 1, 0, 0, 0, 0, 0},
                    {0, 0, 1, 0, 0, 0, 0},
                    {0, 0, 0, 1, 0, 0, 0},
                    {0, 0, 0, 0, 1, 0, 0},
                    {0, 0, 0, 0, 0, 1, 0}
                    };

int shiftPT[2] = {low_rpm, high_rpm};   // sets lower and upper limits of RPMs to display

int prev_range = 10,        // needed for tachometer updating, arbitrary number
    prev_gear = 10,         // needed for gear position updating, arbitrary number
    ledStages[2] = {5, 11}, // this is where each stage of the led strip is set. i.e. from ledStages[0] and ledStages[1] is stage one and so on
    led_pos = 0;            // position of LEDs in sleep mode
long prevBlinkTime = 0;     // timer for warning flash interval

bool warningState = false,  // blink state for shift point flashing
     eng_state = false,     // blink state for engine warning LED
     eop_state = false,     // blink state for oil pressure warning LED
     pos_state = false,     // move LEDs left or right in sleep mode
     enable_state = false;  // flag for sleep mode

void setup()
{
  //Serial.begin(115200);     // comment out all Serials and Strings when not testing
  do
  {
    if (CAN_OK == CAN.begin(CAN_1000KBPS))  // initializes CAN-BUS Shield at specified baud rate.
    {
      //Serial.println("CAN-BUS Shield Initialized!");
      //is_CBS_init = true;
      break;
    }
    else
    {
      //Serial.println("CAN-BUS Shield FAILED!");
      //Serial.println("Retry initializing CAN-BUS Shield.");
      delay(1000);
    }
  }
  while(true);

  pinMode(neutralPin, INPUT); // sets neutralPin (pin 6) as input
  
  // begin NeoPixel strip
  strip.begin();
  strip.setBrightness(stripBrightness);
  strip.clear();
  strip.show();

  // begin segment display
  seg.begin();
  seg.setBrightness(segBrightness);
  seg.clear();
  seg.show();

  LED.begin();
  
  startupAnimation();
}

void loop()
{
  unsigned char len = 0;
  unsigned char buf[4];
  if(CAN_MSGAVAIL == CAN.checkReceive()) //Checks for incoming data.
  {
    CAN.readMsgBuf(&len, buf);           //Reads incoming data. len: data length, buf[location]: actual data.
    unsigned int canId = CAN.getCanId();
    //String id = String(canId);
    //Serial.println(id);
    if (canId == 1536)
    {
      int rpmA = buf[0];
      int rpmB = buf[1];
      int ENGINE_RPM = ((rpmA * 256) + rpmB);
      ledStrip_update(ENGINE_RPM);
      //gearOFF(ENGINE_RPM);
      //String ALPHA_RPM = String(ENGINE_RPM);
      //Serial.println("RPM: " + ALPHA_RPM + "\n");
    }

    if (canId == 1550)
    {
      int gearA = buf[2];
      int gearB = buf[3];
      int GEAR = ((gearA * 256) + gearB - 2);
      gearShift_update(GEAR, segColor);
      //String ALPHA_GEAR = String(GEAR);
      //Serial.println("GEAR: " + ALPHA_GEAR + "\n");
    }
  
    if (canId == 1541)
    {
      int tempA = buf[2];
      int tempB = buf[3];
      int ENGINE_TEMP = ((tempA * 256) + tempB) / 10;
      TEMP_warning(ENGINE_TEMP);
      //String ALPHA_TEMP = String(ENGINE_TEMP);
      //Serial.println("ENG TEMP: " + ALPHA_TEMP + "\n");
    }

    if (canId == 1544)
    {
      int eopA = buf[0];
      int eopB = buf[1];
      int INT_EOP = (((eopA * 256) + eopB) * 0.0145037738); // millibars to PSI conversion
      EOP_warning(INT_EOP);
      //String ALPHA_EOP = String(INT_EOP);
      //Serial.println("EOP: " + ALPHA_EOP + "\n");
    }

    if (canId == 1546)
    {
      int enableA = buf[0];
      int enableB = buf[1];
      int calA = buf[2];
      int calB = buf[3];
      int ENABLE = ((enableA * 256) + enableB);
      int CAL = ((calA * 256) + calB);
      brightness_ctrl(CAL);
      sleep(ENABLE);
      //String ALPHA_EN = String(ENABLE);
      //Serial.println("ENABLE: " + ALPHA_EN + "\n");
      //String ALPHA_CAL = String(CAL);
      //Serial.println("CAL: " + ALPHA_CAL + "\n");
    }
  }
}

void ledStrip_update(int rpm)   // tachometer function
{
  int LED_RPM = rpm;
  unsigned long currentMillis = millis();   // get current timer for flashing condition
  if (LED_RPM >= shiftPT[0] && LED_RPM < shiftPT[1]) // if the RPM is between the lowest RPM and the shift point
  {
    // map the RPM values from 0 to 16(really 15 since the shift point and beyond is handled below) and constrain the range
    int rpmMapped = map(LED_RPM, shiftPT[0], shiftPT[1], 0, 16);
    int rpmConstrained = constrain(rpmMapped, 0, 16);

    if (prev_range != rpmConstrained)                // this makes it so we only update the LED when the range changes so we don't readdress the strip every reading
    {
      prev_range = rpmConstrained;
      strip.clear();
      for (int ledNum = 0; ledNum <= rpmConstrained; ledNum++) // rpmConstrained makes ledNum refer to one LED too much on the light strip since the light strip is indexed from 0,
                                                               // causes brief moment of max rpm before flashing to change gear
      {
        if (ledNum <= ledStages[0])                            // green
        {
          strip.setPixelColor(ledNum, color[0]);
          segColor = greenSeg;
        }
        else if (ledNum > ledStages[0] && ledNum <= ledStages[1])   // yellow
        {
          strip.setPixelColor(ledNum, color[1]);
          segColor = yellowSeg;
        }
        else if (ledNum > ledStages[1] && ledNum < stripLength)     // red
        {
          strip.setPixelColor(ledNum, color[2]);
          segColor = redSeg;
        }
      }
      strip.show();
    }
  }
  else if (LED_RPM >= shiftPT[1]) // RPM at or above shift point, blinks the LEDS on and off with no blocking delay
  {
    prev_range = 16;
    if (currentMillis - prevBlinkTime > blinkInterval)    // millis eventually overflows but only after 50 days powered on so abs value is not needed here
    {
      prevBlinkTime = currentMillis;

      if (warningState == false)
      {
        warningState = true;
        for (int i = 0; i < stripLength; i++)
        {
          strip.setPixelColor(i, color[3]);
        }
        strip.show();
      } 
      else
      {
        strip.clear();
        strip.show();
        warningState = false;
      }
    }
  }
  else
  {
    if (prev_range != 10)
    {
      prev_range = 10;
      strip.clear();
      strip.show();
    }
  }
}

void gearShift_update(int gear, uint32_t segColor) // update and display gear number on segment display
{
  //Serial.println("Neutral:" + digitalRead(neutralPin));
  if (digitalRead(neutralPin) == LOW)
  {
    gear = 0;
    segColor = greenSeg;
    seg.clear();
    seg.show();
    for(int i = 0; i <= 6; i++)
    {
      if(digitArray[gear][i])
        seg.setPixelColor(i, segColor);
    }
    seg.show();
  }

  // short-circuit evaluation optimization
  else if (prev_segColor != segColor || gear != prev_gear) {
    prev_segColor = segColor;
    prev_gear = gear;
    seg.clear();
    seg.show();
    for(int i = 0; i <= 6; i++)
    {
      if(digitArray[gear][i])
        seg.setPixelColor(i, segColor);
    }
    seg.show();
  }
}

void TEMP_warning(int engine_temp) // turn LED on when engine temp is outside safe parameters
{
  if (engine_temp >= HIGH_ENG_TEMP && engine_temp < UNSAFE_ENG_TEMP)
  {
    LED.setPWM(ENG_LED, PWM_LEVEL);
  }
  else if (engine_temp >= UNSAFE_ENG_TEMP)
  {
    if (eng_state == false)
    {
      LED.setPWM(ENG_LED, PWM_LEVEL);
      eng_state = true;
    }
    else
    {
      LED.setPWM(ENG_LED, 0);
      eng_state = false;
    }
  }
  else
  {
    LED.setPWM(ENG_LED, 0);
  }
  LED.write();
}

void EOP_warning(int EOP) // turn LED on when oil pressure is outside safe parameters
{
  if (EOP >= HIGH_EOP)
  {
    LED.setPWM(OIL_LED, PWM_LEVEL);
  }
  else if (EOP <= 8)
  {
    if (eop_state == false)
    {
      LED.setPWM(OIL_LED, PWM_LEVEL);
      eop_state = true;
    }
    else
    {
      LED.setPWM(OIL_LED, 0);
      eop_state = false;
    }
  }
  else
  {
    LED.setPWM(OIL_LED, 0);
  }
  LED.write();
}

void brightness_ctrl(int CAL)
{
  if (CAL >= 0 && CAL <= 3) // day time brightness
  {
    color[0] = greenStripDay;
    color[1] = yellowStripDay;
    color[2] = redStripDay;
    color[3] = redMaxStripDay;

    redSeg = redSegDay;
    yellowSeg = yellowSegDay;
    greenSeg = greenSegDay;
    
    DUTY = 5;
    PWM_LEVEL = map(DUTY, 0, 100, 0, 65535);
  }

  else  // night time brightness
  {
    color[0] = greenStripNight;
    color[1] = yellowStripNight;
    color[2] = redStripNight;
    color[3] = redMaxStripNight;

    redSeg = redSegNight;
    yellowSeg = yellowSegNight;
    greenSeg = greenSegNight;
    
    DUTY = 1;
    PWM_LEVEL = map(DUTY, 0, 100, 0, 65535);
  }
}

void sleep(int ENABLE)  // idle animation when engine is not running
{
  if (ENABLE == 0)
  {
    strip.clear();
    
    if (led_pos <= ledStages[0])
      strip.setPixelColor(led_pos, color[0]);
      
    else if (led_pos > ledStages[0] && led_pos <= ledStages[1])
      strip.setPixelColor(led_pos, color[1]);
      
    else if (led_pos > ledStages[0] && led_pos < stripLength)
      strip.setPixelColor(led_pos, color[2]);
    
    strip.show();

    if (pos_state == false)
    {
      led_pos++;
      if (led_pos == 15)
        pos_state = true;
    }
    else if (pos_state == true)
    {
      led_pos--;
      if (led_pos == 0)
        pos_state = false;
    }

    enable_state = false;
  }

  else if (ENABLE == 1 && enable_state == false)
  {
    strip.clear();
    strip.show();
    enable_state = true;
  }
}

void startupAnimation()
{
  LED.setPWM(ENG_LED, PWM_LEVEL);
  LED.setPWM(OIL_LED, PWM_LEVEL);
  LED.write();
  segColor = greenSeg;
  for(int h = 0; h <= 15; h++)
  {
    for(int i = 0; i <= 5; i++)
    {
      for(int j = 0; j <= 6; j++)
      {
        if(digitCircle[i][j])
          seg.setPixelColor(j, segColor);
      }
      seg.show();
      delay(30);
      seg.clear();
    }

    if (h <= ledStages[0])
    {
      strip.setPixelColor(h, color[0]);
      segColor = greenSeg;
    }
    else if (h > ledStages[0] && h <= ledStages[1])
    {
      strip.setPixelColor(h, color[1]);
      segColor = yellowSeg;
    }
    else if (h > ledStages[1] && h < stripLength)
    {
      strip.setPixelColor(h, color[2]);
      segColor = redSeg;
    }
    strip.show();
  }
  seg.clear();
  seg.show();
  LED.setPWM(ENG_LED, 0);
  LED.setPWM(OIL_LED, 0);
  LED.write();
  delay(500);
  strip.clear();
  strip.show();
  delay(1000);
}